# Algoritmia
## Práctica Obligatoria 3
### Curso 2023 - 2024
###### Programación dinámica
---
 

#### Autores:
* Cristian Fernández Martínez
* Alicia García Pérez

---
Resuelva la siguiente práctica.

Importe las librerías que desees
**Recuerda**: 
* Solamente puedes utilizar librerías nativas (https://docs.python.org/es/3.8/library/index.html)
* Las funciones que importes no son "gratis", cada una tendrá una complejidad temporal y espacial que se tendrá que tener en cuenta.

In [5]:
#testeable
# Imports útiles

In [6]:
#testeable
class Video:
    """
    Clase Video. 
    Representa una serie o película.
    """    
    
    def __init__(self, name, size):
        """Crea un objeto de clase Video

        Parameters
        ----------
        name : str
            Nombre de la serie/película
        size : number
            Tamaño en memoria de la serie/película
        """
        self.name = name
        self.size = size
    
    def __hash__(self):
        """Genera el valor hash identificativo del vídeo

        Returns
        -------
        int
            Valor hash
        """        
        return hash((self.name,self.size))
    
    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """        
        return f'Nombre del video: {self.name}, tamaño: {self.size}'
    
    def __repr__(self):
        """Genera una cadena descriptiva del objeto dentro de colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """  
        return f'Nombre del video: {self.name}, tamaño: {self.size}, hash: {hash(self)}'
       
    def set_users(self, country, users):
        """Dado un pais y un número de usuarios
           almacena para este vídeo la cantidad de espectadores que tiene.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película
        users : int
            Número de espectadores
        """        
        self.users[country]=users
    
    def get_users(self, country):
        """Dado un país, obtiene el número de usuarios.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película

        Returns
        -------
        int
            Número de espectadores para el país `country`
        """       
        return self.users.get(country,0)

    def tiempo_emision(self, country):
        """Dado un país, obtiene el tiempo de emisión.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película

        Returns
        -------
        int
            Número de minutos de emisión para el país `country`
        """   
        return self.get_users(country)*self.size     


In [7]:
#testeable
class ServidorCache:
    """
    Clase del servidor caché donde se almacenan parte de series/películas.
    """
    
    def __init__(self, identifier, country, capacity):
        """Instancia un Servidor de Caché

        Parameters
        ----------
        identifier : int
            Valor que identifica un servidor.
        country : str
            País donde está el servidor.
        capacity : int
            Cantidad de memoria de almacenamiento disponible.
        """
        self.identifier = identifier
        self.country = country
        self.capacity = capacity
        
    def __hash__(self):
        """Genera el valor hash identificativo del servidor

        Returns
        -------
        int
            Valor hash
        """    
        return hash((self.identifier, self.country, self.capacity))

    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """      
        return f'Servidor {self.identifier} en {self.country} con capacidad {self.capacity}'
    
    def __repr__(self):
        """Genera una cadena descriptiva del objeto en colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """      
        return f'Servidor {self.identifier} en {self.country} con capacidad {self.capacity}, hash: {hash(self)}'
    
    def rellena(self, videos):
        """Dada una colección de videos,
           seleccionar aquellos que se van a almacenar en el servidor.
           Se ha de optimizar para que el tiempo de emisión sea el máximo posible.
           No se pueden partir los vídeos.

        Parameters
        ----------
        videos : collection
            Colección de videos que se quieren almacenar en el servidor.
        """
        almacenados = []
        for video in videos:
            video.valor = video.tiempo_emision(self.country)
        
        videos = sorted(videos, key=lambda x: x.valor, reverse=True)

        peso_utilizado = 0
        while peso_utilizado < self.capacity:
            for video in videos:
                if video.size + peso_utilizado <= self.capacity:
                    almacenados.append(video)
                    peso_utilizado += video.size
        
        return almacenados
            
    def tiempo_emision(self):
        """A partir de los datos almacenados
           devolver el tiempo de emisión óptimo del servidor.

        Returns
        -------
        number
            Tiempo de Emision            
        """        
        tiempo_emision = 0
        for video in self.almacenados:
            tiempo_emision += video.tiempo_emision(self.country)

        return tiempo_emision

    def almacenados(self):
        """A partir de los datos almacenados
           devolver los objetos vídeo.

        Returns
        -------
        collection
            Colección de videos almacenados en el servidor.
        """     
        return self.almacenados
    

In [8]:
#testeable
class ServidorMaestro:
    """
    Servidor central que gestiona las conexiones entre servidores cache
    """
    
    def __init__(self, servidores, distancias):
        """Instancia el servidor central

        Parameters
        ----------
        servidores : Iterable
            Conjunto de servidores cache disponibles
        distancias : dict{ServidorCache: dict{ServidorCache: int}}
            Grafo de distancias en milisegundos entre servidores.
        """        
        self.servidores = set(servidores)
        self.distancias = distancias      
          
    def get_grafo(self):
        """Devuelve el grafo de distancias recibido

        Returns
        -------
        dict{ServidorCache: dict{ServidorCache: int}}
            Grafo de distancias en milisegundos entre servidores.
        """        
        return self.distancias

    def calcula_distancias(self):
        """Calcula las distancias MÍNIMAS entre servidores cache
           y los correspondientes caminos.
        """        
        self.caminos = {i: {j: 0 for j in self.servidores} for i in self.servidores}
        self.distancia_minima = self.distancias.copy()

        for intermedio in self.servidores:
            for origen in self.servidores:
                for destino in self.servidores:
                    if self.distancia_minima[origen][destino] > self.distancia_minima[origen][intermedio] + self.distancia_minima[intermedio][destino]:
                        self.distancia_minima[origen][destino] = self.distancia_minima[origen][intermedio] + self.distancia_minima[intermedio][destino]
                        self.caminos[origen][destino] = intermedio
    

    def distancia(self, origen, destino):
        """
        Devuelve la distancia mmochila[j], left + videos[i].tiempo_emision(self.country)ínima entre dos servidores cache.

        Parameters
        ----------
        origen : ServidorCache
            Servidor de origen
        destino : ServidorCache
            Servidor de destino

        Returns
        -------
        int
            Distancia mínima en milisegundos entre los servidores.
        """
        if not hasattr(self, 'distancia_minima'):
            self.calcula_distancias()        
        return self.distancia_minima[origen][destino]

    def camino(self, origen, destino):
        """
        Devuelve el camino mínimo entre dos servidores cache.

        Parameters
        ----------
        origen : ServidorCache
            Servidor de origen
        destino : ServidorCache
            Servidor de destino

        Returns
        -------
        list<ServidorCache>
            Lista de servidores para llegar de origen a destino. 
            Se debe incluir al origen y al destino.
        """
        if not hasattr(self, 'caminos'):
            self.calcula_distancias()
        camino = [origen]
        while origen != destino:
            origen = self.caminos[origen][destino]
            camino.append(origen)
        return camino

### Pruebas de ejemplo

In [9]:
import unittest
import json

def carga_dataset(data):
    with open(data) as f:
        test_datasets = json.load(f)

    videos = list()
    for v in test_datasets["videos"]:
        v_obj = Video(v["name"], v["size"])
        for c, u in v["users"].items():
            v_obj.set_users(c, u)
        videos.append(v_obj)

    servers = dict()
    for s in test_datasets["servers"]:
        servers[s["country"]] = ServidorCache(s["identifier"], s["country"], s["size"])


    pings = test_datasets["pings"]
    p_ = dict()
    for s in servers.values():
        p_[s] = dict()
        for p in pings[s.country]:
            ping = pings[s.country][p]
            if pings[s.country][p] == -1:
                ping = float("inf")
            p_[s][servers[p]] = ping
    maestro = ServidorMaestro(servers.values(), p_)

    return videos, servers, maestro

class TestBasico(unittest.TestCase):
    
    def test_carga_simple(self):
        
        v, s, m = carga_dataset("toy_PD.json")

        spain = s["Spain"]
        spain.rellena(v)
        self.assertEqual(spain.tiempo_emision(), 55800)
        almacenados = spain.almacenados()
        self.assertEqual(len(almacenados), 3)

        m.calcula_distancias()
        self.assertEqual(m.camino(s["France"], s["Ireland"]), [s["France"], s["Spain"], s["Ireland"]])
        self.assertEqual(m.camino(s["France"], s["Ireland"]), [s["France"], s["Spain"], s["Ireland"]])
        self.assertEqual(m.distancia(s["France"], s["Ireland"]), 200)      
        

In [10]:
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

E
ERROR: test_carga_simple (__main__.TestBasico.test_carga_simple)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\crist\AppData\Local\Temp\ipykernel_3464\3849330748.py", line 37, in test_carga_simple
    v, s, m = carga_dataset("toy_PD.json")
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\crist\AppData\Local\Temp\ipykernel_3464\3849330748.py", line 12, in carga_dataset
    v_obj.set_users(c, u)
  File "C:\Users\crist\AppData\Local\Temp\ipykernel_3464\3116030752.py", line 62, in set_users
    self.users[country]=users
    ^^^^^^^^^^
AttributeError: 'Video' object has no attribute 'users'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


##### **Tests**

Para probar que tu solución pasa los tests. Utilice el comando:

```bash
$ python tests-py3<version de python> <mi notebook>
```

Los tests necesitan de la librería `nbformat`

```bash
$ pip install nbformat
```

###### Explicación de los tests
* `test_ej1_tiempo_emision_correcto`: Comprueba que el tiempo de emisión es el máximo para cada servidor.
* `test_ej1_emision_maxima`: Comprueba que ante un servidor virtualmente infinita el tiempo de emisión es el máximo posible.
* `test_ej1_pais_no_existe`: Comprueba que el tiempo de emisión de un país que no existe es 0.
* `test_ej2_distancias_correctas`: Comprueba que la distancia es la mínima posible.
* `test_ej2_caminos_correctos`: Comprueba que los caminos son los mínimos.
* `test_ej2_camino_nulo`: Comprueba que ante un viaje cuyo origen es igual al destino el camino es solo el nodo y el coste es 0.

---

### **Informe**
Contesta a las siguientes preguntas (Justificando las respuestas).

#### **Complejidad**

1. Método `ServidorCache.rellena`
    * **Complejidad temporal**: _contesta aquí_
2. Método `ServidorMaestro.calcula_distancias`
    * **Complejidad temporal**: _contesta aquí_


#### **Respecto a la maximización del tiempo de emisión del servidor caché.**

* ¿Cómo de buena es la solución? ¿Es óptima o aproximada?<br><sub style="font-style: italic">No confundir una solución óptima (obtiene el máximo posible) frente a una solución eficiente (calcula el máximo con la menor complejidad posible).</sub>

_Contesta aquí justificando la respuesta_

* ¿Cómo cambiaría la complejidad espacial si aumentase el tamaño del servidor? ¿Y si aumentase el número de vídeos?

_Contesta aquí justificando la respuesta_

#### **Respecto a la búsqueda de distancias mínimas entre servidores.**

* Este problema tiene una aproximación voraz, ¿cuál sería la ventaja de usarlo frente a la solución que has planteado? 

_Contesta aquí justificando la respuesta_

* ¿Y cuál sería su desventaja?

_Contesta aquí justificando la respuesta_